# Create Tiles for Indonesia Coastline

In [1]:
import os

import geopandas as gpd
import pandas as pd
from odc.geo import BoundingBox
from ipyleaflet import basemaps

from coastlines.grids import INDONESIA_25, INDONESIA_10, INDONESIA_CRS

from odc.geo.gridspec import GridSpec

os.environ["USE_PYGEOS"] = "0"

## Shows which coastlines.grids library is imported

In [2]:
import coastlines.grids
print(coastlines.grids.__file__)

/opt/conda/lib/python3.12/site-packages/coastlines/grids.py


It looks like the imported libraries are not from this repository, so we need to define the grid in this notebook and take some parameters from the existing grids.
First shows grid parameters.

In [3]:
print('INDONESIA_25:', INDONESIA_25, INDONESIA_25.origin)
print('INDONESIA_10:', INDONESIA_10, INDONESIA_10.origin)
print('INDONESIA_CRS:', INDONESIA_CRS)

INDONESIA_25: GridSpec(crs=EPSG:3832, tile_shape=Shape2d(x=2000, y=2000), resolution=Resolution(x=25, y=-25)) XY(x=-6250000, y=-1250000)
INDONESIA_10: GridSpec(crs=EPSG:3832, tile_shape=Shape2d(x=5000, y=5000), resolution=Resolution(x=10, y=-10)) XY(x=-6250000, y=-1250000)
INDONESIA_CRS: EPSG:3832


## Create Indonesia 10 km tiles

Create new Indonesia grid with 10 km tiles using GridSpec

In [4]:
INDONESIA_10KM = GridSpec(
    crs='ESRI:54034', # Cylindrical Equal Area
    tile_shape=(5000, 5000),
    resolution=2,
    origin=INDONESIA_25.origin,
)
print('INDONESIA_10KM:', INDONESIA_10KM, INDONESIA_10KM.origin)

INDONESIA_10KM: GridSpec(crs=ESRI:54034, tile_shape=Shape2d(x=5000, y=5000), resolution=Resolution(x=2, y=-2)) XY(x=-6250000, y=-1250000)


Create tiles with INDONESIA_10KM grid specs

In [5]:
# Select relevant tiles and convert to Geopandas
bounds = BoundingBox(94, -11, 142, 6, crs='EPSG:4326').to_crs(INDONESIA_10KM.crs)
geom = INDONESIA_10KM.geojson(bbox=bounds)
tiles_projected_10km = gpd.GeoDataFrame.from_features(geom, crs='EPSG:4326').to_crs(
    INDONESIA_10KM.crs
)

## Indonesia coast tiles

Read Indonesia's 5K coastline

In [6]:
idn_coastline = gpd.read_file('../data/raw/GARISPANTAI_5K_SEPT2024.gdb')

Check if Indonesia 10km tiles has the same CRS as the coastline data

In [7]:
if tiles_projected_10km.crs != idn_coastline.crs:
    print(f'Reproject tiles to {idn_coastline.crs}') 
    tiles_idn = tiles_projected_10km.to_crs(crs=str(idn_coastline.crs).upper())
else:
    print('Same CRS')
    tiles_idn = tiles_projected_10km.copy()

Reproject tiles to EPSG:4326


Perform spatial join: tiles that intersect any coastline segment. Tiles id column merged into the table. 

In [8]:
# Perform spatial join: tiles that intersect any coastline segment
tiles_idn_coast = gpd.sjoin(tiles_idn, idn_coastline, predicate="intersects", how="inner")
tiles_idn_coast.head(3)

,geometry,idx,index_right,NAMOBJ,FCODE,REMARK,METADATA,SRS_ID,DTMVER,ELEVAS,KARGPN,TIPGPN,KODGPN,THNSBDATA,SBDATA,SKL,THNPBL,KET,Panjang,Shape_Length
321,"POLYGON ((122.7997 -10.91663, 122.7997 -11.008...","1992,4",34112,Garis Pantai,DB02080140,Digitasi - FU,,4326,NaN,NaN,5.0,4,,2011-2014,6.0,4,2024,,12.584924,0.113479
321,"POLYGON ((122.7997 -10.91663, 122.7997 -11.008...","1992,4",33889,Garis Pantai,DB02080140,Digitasi - FU,,4326,NaN,NaN,3.0,4,,2011-2014,6.0,4,2024,,0.331914,0.002994
321,"POLYGON ((122.7997 -10.91663, 122.7997 -11.008...","1992,4",34113,Garis Pantai,DB02080140,Digitasi - FU,,4326,NaN,NaN,3.0,4,,2011-2014,6.0,4,2024,,0.791579,0.007135


Keep unique tiles only. Removing redundant tiles and unnecessary columns.

In [9]:
# Keep unique tiles only
tiles_idn_coast = tiles_idn.loc[tiles_idn.index.isin(tiles_idn_coast.index)]
tiles_idn_coast.head(3)

,geometry,idx
321,"POLYGON ((122.7997 -10.91663, 122.7997 -11.008...","1992,4"
322,"POLYGON ((122.88953 -10.91663, 122.88953 -11.0...","1993,4"
323,"POLYGON ((122.97936 -10.91663, 122.97936 -11.0...","1994,4"


Filtered tiles based on coastline data.

In [10]:
tiles_idn_coast.explore()

In [11]:
# Export the results as Geopackage and GeoJSON
tiles_idn_coast = tiles_idn_coast.rename(columns={'idx': 'id'})
tiles_idn_coast.to_crs('EPSG:4326').to_file(
    '../data/raw/indonesia_10km_tiles_coast.geojson', driver='GeoJSON'
)
tiles_idn_coast.to_crs('EPSG:4326').to_file(
    '../data/raw/indonesia_10km_tiles_coast.gpkg', driver='GPKG'
)